In [ ]:
from transformers import pipeline
from PIL import Image, ImageChops
import requests
import numpy as np
import scipy
import cv2
import os

from moviepy.editor import ImageSequenceClip
from IPython.display import display

SAMPLE_FRAME = "sample.png"

### Step 1: Download a depth estimation model

In [ ]:
PIPE = None

def get_pipe():
    # This will load the pipeline on demand on the current PROCESS/THREAD.
    # And load it only once.
    global PIPE
    if PIPE is None:
        PIPE = pipeline(task="depth-estimation", model="LiheYoung/depth-anything-large-hf")
    return PIPE

### Step 2. Shift an image

In [ ]:
def shift_image(data, depth_img, shift_amount=10):
    # Ensure depth image is grayscale (for single value)
    data_with_alpha = np.dstack([data, np.full(data.shape[:2], 255, dtype=np.uint8)])
    frame = Image.fromarray(data_with_alpha, "RGBA")
    frame_depth = get_pipe()(frame)["depth"]
    depth_img = frame_depth.convert("L")
    depth_data = np.array(depth_img)
    deltas = np.array((depth_data / 255.0) * float(shift_amount), dtype=int)

    # This creates the transprent resulting image.
    # For now, we're dealing with pixel data.
    shifted_data = np.zeros_like(data_with_alpha)

    width = frame.width
    height = frame.height

    for y, row in enumerate(deltas):
        width = len(row)
        x = 0
        while x < width:
            dx = row[x]
            if x+dx >= width:
                break
            if x-dx < 0:
                shifted_data[y][x-dx] = [0,0,0,0]
            else:
                shifted_data[y][x-dx] = data_with_alpha[y][x]
            x += 1

    # Convert the pixel data to an image.
    shifted_image = Image.fromarray(shifted_data)

    alphas_image = Image.fromarray(scipy.ndimage.binary_fill_holes(ImageChops.invert(shifted_image.getchannel("A")))).convert("1")
    shifted_image.putalpha(ImageChops.invert(alphas_image))
    return shifted_image

In [ ]:
def shift_and_inpaint(img, amount):
    shifted = shift_image(img, depth, shift_amount=amount).convert('RGB')
    org_img = np.array(shifted)
    damaged_img = np.array(shifted)
    
    # get the shape of the image
    height, width = damaged_img.shape[0], damaged_img.shape[1]
     
    # Converting all pixels greater than zero to black while black becomes white
    for i in range(height):
        for j in range(width):
            if damaged_img[i, j].sum() > 0:
                damaged_img[i, j] = 0
            else:
                damaged_img[i, j] = [255, 255, 255]
     
    # saving the mask 
    mask = cv2.cvtColor(damaged_img, cv2.COLOR_BGR2GRAY)
    dst = cv2.inpaint(org_img, mask, 3, cv2.INPAINT_NS)
    return dst

In [ ]:
def create_over_under_video_frame(frame):
    idx, image = frame
    print(idx)
    left_img = shift_and_inpaint(image, 10)
    right_img = shift_and_inpaint(image, 50)
    
    height, width, channels = left_img.shape
    
    stacked_img = cv2.vconcat([left_img, right_img])  # Combine images side by side
    stacked_img = cv2.cvtColor(stacked_img, cv2.COLOR_BGR2RGB)
    return (idx, stacked_img)

### Load a video and grab individual frames from the video

In [6]:
def read_frames(path):
    cap = cv2.VideoCapture(VIDEO_PATH)
    frame_idx = 0

    while True:
        frame_idx += 1
        
        if frame_idx == 10:
            break
            
        # Read a new frame
        ret, frame = cap.read()

        # If frame is read correctly ret is True
        if not ret:
            print("Can't receive frame (stream end?). Exiting ...")
            break

        yield (frame, frame_idx)

In [ ]:
from torch.multiprocessing import Pool, Process, set_start_method

if __name__ == "__main__":
    result = []
    set_start_method("spawn", force=True)

    VIDEO_PATH = "sample.mov"
    cap = cv2.VideoCapture(VIDEO_PATH)
    frame_id = 0

    multi_pool = Pool(processes=3)
    frames = list(read_frames(VIDEO_PATH))
    output = multi_pool.map(create_over_under_video_frame, list(frames[0]))
    multi_pool.close()
    multi_pool.join()

    # fps = 30
    # sorted_frames = sorted(frames, key=lambda x : x[0])
    # clip = ImageSequenceClip(sorted_frames, fps=fps)
    # video_path = 'sbs_3d_video.mp4'
    # clip.write_videofile(video_path, codec='libx264', audio=False)

Process SpawnPoolWorker-1:
Process SpawnPoolWorker-2:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/opt/homebrew/Cellar/python@3.10/3.10.13_2/Frameworks/Python.framework/Versions/3.10/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/opt/homebrew/Cellar/python@3.10/3.10.13_2/Frameworks/Python.framework/Versions/3.10/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/homebrew/Cellar/python@3.10/3.10.13_2/Frameworks/Python.framework/Versions/3.10/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/opt/homebrew/Cellar/python@3.10/3.10.13_2/Frameworks/Python.framework/Versions/3.10/lib/python3.10/multiprocessing/pool.py", line 114, in worker
    task = get()
  File "/opt/homebrew/Cellar/python@3.10/3.10.13_2/Frameworks/Python.framework/Versions/3.10/lib/python3.10/multiprocessing/queues.py", line 367, in get


In [ ]:
# 